In [1]:
import pandas as pd
import xml.etree.ElementTree as ET
from pymongo import MongoClient
import json

In [2]:
file_path = 'Electric_Vehicle_Population_Data.xml'

def read_xml_to_df(file_path):
    # Parse XML data and get root element
    tree = ET.parse(file_path)
    root = tree.getroot()
    
    data = []
    # find the row element which is nested inside another row element
    for outer_row_element in root.findall('.//row'):
        row_data = {}
        for inner_row_element in outer_row_element.findall('.//row'):
            for child_element in inner_row_element:
                row_data[child_element.tag] = child_element.text
            data.append(row_data.copy())

    # Convert the data into dataframe
    return pd.DataFrame(data)

df = read_xml_to_df(file_path)

In [4]:
# function to insert the data into mongo db
def insert_to_mongoDB(url, dataframe, database, collection):

    client = MongoClient(url)

    db = client[database]
    collection = db[collection]

    # Convert DataFrame to JSON
    json_data = dataframe.to_json(orient='records')

    # Insert Data into MongoDB
    data = json.loads(json_data)
    result = collection.insert_many(data)

    # Print confirmation
    if result.inserted_ids:
        print(f"Data inserted successfully")
    else:
        print("Failed to insert data")

mongo_url = 'mongodb://localhost:27017/'
database = 'Washington_State_Department_of_Licensing'
collection = 'Electric_Vehicle_Population'
insert_to_mongoDB(mongo_url, df, database, collection)

Data inserted successfully
